In [1]:
# Misc imports
import os
import json
import pandas as pd

# Import Minions + Minions
from minions.minions import Minions
from minions.minion import Minion

# Import Minion Clients
from minions.clients.ollama import OllamaClient
from minions.clients.tokasaurus import TokasaurusClient
from minions.clients.openai import OpenAIClient
from minions.clients.anthropic import AnthropicClient
from minions.clients.together import TogetherClient

# Import Pydantic
from pydantic import BaseModel

### Configure Clients


1. Specify structured output schema for the OllamaClient. Run this block as is! Do _NOT_ make any modifications


In [2]:
class StructuredLocalOutput(BaseModel):
    explanation: str
    citation: str | None
    answer: str | None

2. Instantiate Clients: Here we instantiate our local client to be ollama and remote client to be OpenAI


In [ ]:
remote_client = OpenAIClient(model_name="gpt-4o", temperature=0.0)

# Option 1: Ollama
local_client = OllamaClient(
    model_name="llama3.2:3b", 
    temperature=0.0, 
    structured_output_schema=StructuredLocalOutput
)

# option 2
# local_client = TokasaurusClient(
#     model_name="meta-llama/Llama-3.2-1B-Instruct",
#     temperature=0.0,
#     # Structured outputs are not yet supported with tokasaurus
#     # structured_output_schema=StructuredLocalOutput
# )



Starting server with command: tksrs             port=57475             model=meta-llama/Llama-3.2-1B-Instruct             torch_compile=T             
Waiting for server to start on port 57475...
ConnectionError: HTTPConnectionPool(host='localhost', port=57475): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd732122e10>: Failed to establish a new connection: [Errno 111] Connection refused'))
Server not yet started (attempt 0) retrying... (this is normal)
ConnectionError: HTTPConnectionPool(host='localhost', port=57475): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd738635640>: Failed to establish a new connection: [Errno 111] Connection refused'))
Server not yet started (attempt 1) retrying... (this is normal)
Setting max_num_tokens_per_request to 131072
Starting 3 processes: ['model_worker', 'manager', 'server']
Running in the main process: 

Loading safetensors files: 100%|██████████| 1/1 [00:00<00:00,  2.82it/s]
INFO:     Started server process [27630]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:57475 (Press CTRL+C to quit)


2025-02-25 06:39:47 | INFO | model_worker | Created model
System startup time: 4.850164175033569
2025-02-25 06:39:47 | INFO | model_worker | Model loop started!
              _              _                       
  ___   __ _ | |_       ___ | |  ___   ___  _ __     
 / _ \ / _` || __|     / __|| | / _ \ / _ \| '_ \    
|  __/| (_| || |_  _   \__ \| ||  __/|  __/| |_) | _ 
 \___| \__,_| \__|( )  |___/|_| \___| \___|| .__/ ( )
                  |/                       |_|    |/ 
 _          __                                                                           _      
(_) _ __   / _|  ___  _ __   ___  _ __    ___   ___      _ __   ___  _ __    ___   __ _ | |_    
| || '_ \ | |_  / _ \| '__| / _ \| '_ \  / __| / _ \    | '__| / _ \| '_ \  / _ \ / _` || __|   
| || | | ||  _||  __/| |   |  __/| | | || (__ |  __/ _  | |   |  __/| |_) ||  __/| (_| || |_  _ 
|_||_| |_||_|   \___||_|    \___||_| |_| \___| \___|( ) |_|    \___|| .__/  \___| \__,_| \__|(_)
                              

### Set-up Communication Protocol


In [4]:
# Option 1: Minions
# protocol = Minions(local_client=local_client, remote_client=remote_client)

# Option 2: Minion
protocol = Minion(local_client=local_client, remote_client=remote_client)
    

### Configure Minion/Minions Task

- context (List[str]): context that the minions need to reason over
- doc_metadata (str): every task is parameterized by doc_metadata which describes the "type" of information contained in the context
- task (str): description of the query to be completed over the context


##### Specify Input Context


In [5]:
context = """\
Patient Name: John A. Doe
Medical Record Number: 001234567
DOB: 1967-08-22
Gender: Male
Admission Date: 2025-01-15
Discharge Date: 2025-01-20

Chief Complaint:
The patient presented with chest discomfort, shortness of breath, and fatigue. Symptoms began gradually 48 hours before admission and intensified over time.

History of Present Illness:
John A. Doe, a 57-year-old male with a history of type 2 diabetes mellitus and hypertension, was admitted following several episodes of atypical chest pain. He experienced intermittent chest tightness and dyspnea on exertion. The pain did not radiate, and there was no associated nausea.

Past Medical History:
- Type 2 Diabetes Mellitus (diagnosed 2010) – managed with metformin.
- Hypertension (diagnosed 2012) – treated with lisinopril.
- Hyperlipidemia – treated with atorvastatin.
- Former smoker (quit 2015).
- No known drug allergies.

Medications on Admission:
- Metformin 1000 mg twice daily.
- Lisinopril 20 mg daily.
- Atorvastatin 40 mg nightly.
- Aspirin 81 mg daily.

Physical Examination:
General: The patient is alert and oriented, in moderate distress.
Cardiovascular: Blood pressure 150/95 mmHg; heart rate 88 bpm; regular rhythm.
Respiratory: Lungs are clear to auscultation bilaterally; respiratory rate 16 bpm.
Neurological: No focal deficits observed.

Laboratory and Diagnostic Findings (Admission):
- Complete Blood Count: WBC 7.2 x10^9/L, Hemoglobin 13.5 g/dL.
- Basic Metabolic Panel: Serum creatinine 1.0 mg/dL; electrolytes normal.
- Fasting Blood Glucose: 180 mg/dL.
- Hemoglobin A1C: 8.5%.
- Lipid Profile: Total Cholesterol 220 mg/dL, LDL 140 mg/dL, HDL 35 mg/dL.
- ECG: Non-specific ST-T changes.
- Chest X-ray: No acute findings.

Progress Notes:
Day 2: Initiated subcutaneous insulin to complement oral hypoglycemics. Blood pressure remained elevated at 148/92 mmHg.
Day 3: Follow-up labs showed a minor improvement in glycemic levels and stable kidney function.
Day 5: Vital signs improved with a blood pressure of 140/85 mmHg and heart rate of 80 bpm. A repeat lab indicated that the fasting blood glucose had decreased significantly compared to admission.
Day 6: Endocrinology recommended maintaining current treatment with a possible gradual reduction in insulin if improvements continue.
Day 7: Patient stabilized and was discharged with clear instructions for follow-up with both primary care and endocrinology.

Discharge Summary:
John A. Doe was discharged on Day 7 after noticeable improvements in glycemic control and blood pressure. The patient is to continue current medications with a scheduled outpatient review.
Medications at Discharge:
- Metformin 1000 mg BID.
- Lisinopril 20 mg daily.
- Atorvastatin 40 mg nightly.
- Adjusted insulin regimen based on recent glycemic trends.

Additional Information:
- Admission weight: 95 kg (BMI: 31.2).
- Dietary consult completed; patient advised on a diabetic diet.
- Patient educated on adherence and regular blood glucose monitoring.

Summary:
This electronic health record details John A. Doe's hospital admission, clinical findings, laboratory values, treatments, and progress over a 7-day period. Key values include a fasting blood glucose of 180 mg/dL on admission and 150 mg/dL on Day 5, indicating a notable improvement in glycemic control.
"""

##### Specify Metadata and Task Description


In [6]:
doc_metadata = "Patient Visit Notes"

# Define the task without explicitly providing the key values.
task = (
    "Using the electronic health records provided, calculate the percentage decrease in the patient's fasting "
    "blood glucose level from admission to Day 5. Extract the necessary values from the record, show your calculations, "
    "and provide the final percentage decrease."
)

# For testing purposes, we suggest that the correct answer is:
suggested_final_answer = (
    "16.67% decrease in fasting blood glucose from admission to Day 5."
)

### Run the protocol


In [7]:
output = protocol(
        task=task,
        doc_metadata=doc_metadata,
        context=[context],
        max_rounds=5,  # you can adjust rounds as needed for testing
    )

entering loop with max_rounds:  5
getting worker's response


/home/ubuntu/miniconda3/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


2025-02-25 06:40:01 | INFO | manager | ... Max-sized batch complete in 13.61s
2025-02-25 06:40:01 | INFO | manager | Submitting more batches to try to trigger recompiles...
2025-02-25 06:40:37 | INFO | manager | ... Compilation batches complete in 36.55s
2025-02-25 06:40:37 | INFO | manager | Starting manager loop
INFO:     127.0.0.1:50792 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [8]:
print(output['final_answer'])

The patient's fasting blood glucose level decreased by 16.7% from admission to Day 5.
